In [1]:
from sort import *

In [2]:
import cvzone
import cv2
from ultralytics import YOLO
import math
import numpy as np

In [3]:
# cap = cv2.VideoCapture(0) # for web cam 
# cap.set(3, 1280) cant set the size 
# cap.set(4,  720)

cap = cv2.VideoCapture(r"C:\Users\SJK\Videos\object detection videos\car2.mp4")
model = YOLO('yolov8l.pt') # we can also doit with diferent weights

class_names = [
    "Person", "Bicycle", "Car", "Motorcycle", "Airplane", "Bus", "Train", "Truck", "Boat", 
    "Traffic light", "Fire hydrant", "Stop sign", "Parking meter", "Bench", "Bird", "Cat", 
    "Dog", "Horse", "Sheep", "Cow", "Elephant", "Bear", "Zebra", "Giraffe", "Backpack", 
    "Umbrella", "Handbag", "Tie", "Suitcase", "Frisbee", "Skis", "Snowboard", "Sports ball", 
    "Kite", "Baseball bat", "Baseball glove", "Skateboard", "Surfboard", "Tennis racket", 
    "Bottle", "Wine glass", "Cup", "Fork", "Knife", "Spoon", "Bowl", "Banana", "Apple", 
    "Sandwich", "Orange", "Broccoli", "Carrot", "Hot dog", "Pizza", "Donut", "Cake", 
    "Chair", "Couch", "Potted plant", "Bed", "Dining table", "Toilet", "TV", "Laptop", 
    "Mouse", "Remote", "Keyboard", "Cell phone", "Microwave", "Oven", "Toaster", "Sink", 
    "Refrigerator", "Book", "Clock", "Vase", "Scissors", "Teddy bear", "Hair drier", 
    "Toothbrush"
]

mask = cv2.imread(r"C:\Users\SJK\Documents\yolo images\r.png")
mask1 = cv2.resize(mask,(852,480))
print(mask1.shape)

# Tracker
tracker = Sort(max_age= 20 ,min_hits =  3, iou_threshold = 0.3)
limit = [150,250,790,240]
total_count = []


while True:
    suc,frame = cap.read()
    img_region = cv2.bitwise_and(frame,mask1)
    img_graphics = cv2.imread(r"C:\Users\SJK\Documents\yolo images\graphics.png",cv2.IMREAD_UNCHANGED)
    frame = cvzone.overlayPNG(frame,img_graphics,(0,0))
    
    results = model(img_region,stream = True)
    detections = np.empty((0,5))
    
    # creating the bboxes 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # bbox
            x1, y1, x2, y2 = box.xyxy[0] # xyxy islike (x,y,w and h)
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert the val to int
            #print(x1, y1, x2, y2)
            #cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            
            w ,h = x2-x1,y2-y1 # creating w and h for putting rectangle
            
            #----confidence level
            conf = math.ceil(box.conf[0]*100) / 100 # to convert to 2 decimal places
            print(conf)
            

            # ---class name
            cls = int(box.cls[0]) # it gives the class id
            current_class = class_names[cls] # gives the classnames

            if current_class == "Car" or current_class == "Motorcycle" or current_class == "Bus" or current_class == "Truck"  and conf > 0.3:

            
                #cvzone.putTextRect(frame,f'{current_class} {conf}',(max(0,x1),max(30,y1)),scale =0.8,thickness = 1,colorT = (0,0,255), 
                               #colorR = (255,255,255),font =1,offset = 2 )  # to get the where to show the confidence level
                #cvzone.cornerRect(frame,(x1,y1,w,h),colorR=(0, 255, 0),colorC=(0, 0, 255),t=3,rt=2,l=5)

                current_array = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((current_array,detections))


    


    tracker_result = tracker.update(detections)
    cv2.line(frame,(limit[0],limit[1]),(limit[2],limit[3]),(0,0,255),5)
    
    for result in tracker_result:
        x1, y1, x2, y2, id = result
        print(result)
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w ,h = x2-x1, y2-y1
        cvzone.cornerRect(frame,(x1,y1,w,h),colorR=(255,0,0),colorC=(0, 0, 255),t=3,rt=2,l=5)
        cvzone.putTextRect(frame,f'{int(id)}',(max(0,x1),max(30,y1)),scale =2,thickness = 1,colorT = (0,0,255), 
                               colorR = (255,255,255),font =1,offset = 3 )

        centre_x , centre_y = x1+w//2, y1+h//2
        cv2.circle(frame,(centre_x , centre_y),7,(0,255,0),cv2.FILLED)

        if limit[0] < centre_x < limit[2] and limit[1] - 15 < centre_y < limit[1] + 15:
            if total_count.count(id) == 0:
                total_count.append(id)
                cv2.line(frame,(limit[0],limit[1]),(limit[2],limit[3]),(0,255,0),5)
            
    #cvzone.putTextRect(frame,f'total count:{len(total_count)}',(50,50),scale =2,thickness = 1,colorT = (0,0,255), 
                               #colorR = (255,255,255),font =1,offset = 3 )
    cv2.putText(frame,str(len(total_count)),(230,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
    cv2.putText(frame,"Vechile Count",(150,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
        
    
    cv2.imshow("image",frame)
    #cv2.imshow('imgregion',img_region)
    if cv2.waitKey(1) & 0XFF == ord('-'):
        break
cap.release()
cv2.destroyAllWindows()

(480, 852, 3)

0: 384x640 4 cars, 1216.4ms
0.92
0.92
0.9
0.56
Speed: 14.5ms preprocess, 1216.4ms inference, 24.0ms postprocess per image at shape (1, 3, 384, 640)
[        645         274         780         353           4]
[        435         151         531         225           3]
[        234         157         329         220           2]
[        225         158         249         188           1]

0: 384x640 4 cars, 1073.6ms
0.93
0.91
0.9
0.56
Speed: 3.3ms preprocess, 1073.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[     649.13      276.92      779.87      353.08           4]
[     437.85      151.12      534.15      225.88           3]
[     235.06      156.96      331.94      221.04           2]
[     225.91      158.12      250.09      188.88           1]

0: 384x640 4 cars, 918.4ms
0.93
0.91
0.87
0.56
Speed: 6.0ms preprocess, 918.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[     649.58      277.26      779.66      352.92